In [2]:
# [[x,y],[r,g,b],clNo.,distFromClCenter]
import numpy as np
import pandas as pd
import cv2
import scipy
from scipy.spatial import distance
import matplotlib.pyplot as plt
import random
import time

#=========================================================================================
def getDistance(u,v,Pts,CH):
    #dist=Pts[v][3]+distance.euclidean(Pts[CH[Pts[v][2]]][1],Pts[u][1]) #cumulative distance
    dist=distance.euclidean(Pts[CH[Pts[v][2]]][1],Pts[u][1]) #direct distance
    return dist

#=========================================================================================

noOfClusters=4
max_iter_no=5
timeTotalStart=time.time()
input_I = cv2.imread('bream_in_basin.png');
height, width, channels = input_I.shape;

datasize=height*width;

points=[]
for i in range(0,height):
    for j in range(0,width):
        pt=[[i,j],[-1,-1,-1],-1,np.Inf,(i*width)+j]
        for k in range(0,channels):
            pt[1][k]=input_I[i][j][k]
        points.append(pt)

Q=[]
clusters=[]
clusterHeads=[]

clusterColours=[]
for i in range(0,noOfClusters):
    clusterColours.append(np.random.randint(0, 255, size=3))

randomIndices=np.random.randint(0, (datasize-1), size=noOfClusters)
#randomIndices=[8130,8072,8194]

#initialize for first iteration
for i in range(0,noOfClusters):
    points[randomIndices[i]][2]=i
    points[randomIndices[i]][3]=0
    clusterHeads.append(randomIndices[i])
    aHead=[]
    aHead.append(randomIndices[i])
    clusters.append(aHead)   
    
print('111 clusterHeads = ',clusterHeads)

iter_no=0
while True:
    #==cluster building==================================================================
    timestart=time.time()
    # insert neighbour pixels (not fixed) into Q
    for p in points:
        if p[4] not in clusterHeads:
            d=np.Inf
            c=-1
            for ch in clusterHeads:
                if d>distance.euclidean(p[1],points[ch][1]):
                    d=distance.euclidean(p[1],points[ch][1])
                    c=clusterHeads.index(ch)
            p[2]=c
            p[3]=d
            clusters[c].append(p[4])
            
    timeend=time.time()
    print('iteration:',iter_no+1,'cluster building time:',timeend-timestart,'sec')
    #----------------------------------------------------------------------------------------
            
    #==display clustered_image===============================================================       
    clustered_image=np.zeros((height,width,channels), np.uint8)
    for i in range(0,height*width):
        p=int(i/width);
        q=int(i-(p*width));    

        #centerPoint=clusterHeads[ points[i][2]];
        #centerPointp=int(centerPoint/width);
        #centerPointq=int(centerPoint-(centerPointp*width));

        for j in range(0,3):
            clustered_image[p][q][j]=clusterColours[points[i][2]][j]
            #clustered_image[p][q][j]=input_I[centerPointp][centerPointq][j];
            
    #cv2.imshow('clustered_image',clustered_image)
    #cv2.waitKey(2000)
    #cv2.destroyAllWindows()
    #----------------------------------------------------------------------------------------
    
    #==break condition (max iteration)=======================================================
    iter_no=iter_no+1;
    if iter_no==max_iter_no:
        break
    #----------------------------------------------------------------------------------------
    
    #==determining next centers==============================================================
    timestart=time.time()
    #prevPts=points    
    prevHeads=clusterHeads
    print('222 prevHeads = ',prevHeads)
    clusterHeads=[]
    
    for c in range (0,len(clusters)):
        Colour_sum=[0,0,0]
        for p in clusters[c]:
            for ind in range(0,3):
                Colour_sum[ind]=Colour_sum[ind]+points[p][1][ind]
                
        Colour_mean=[x/len(clusters[c]) for x in Colour_sum]
                
        minColour=np.inf
        for p in clusters[c]:
            colour_d=distance.euclidean(points[p][1],Colour_mean)
            if colour_d<minColour:
                newHead_c=p
        
        for p in clusters[c]:
            if p!=newHead_c:
                points[p][2]=-1
                points[p][3]=np.Inf
            else:
                points[p][3]=0    
            
        clusters[c]=[]
        clusters[c].append(newHead_c)
        clusterHeads.append(clusters[c][0])

    print('222 clusterHeads = ',clusterHeads)
    timeend=time.time()   
    print('iteration:',iter_no,'determining next centers time:',timeend-timestart,'sec')
    #----------------------------------------------------------------------------------------
        
    #==break condition (same clusterHead)====================================================
    if clusterHeads==prevHeads:
        print('CONVERGED')
        break
    print('iter_no',iter_no,'done')  
    #----------------------------------------------------------------------------------------
         
    #for p in points:
        #print(p)
    #----------------------------------------------------------------------------------------

timeTotalEnd=time.time()
print('total time:',timeTotalEnd-timeTotalStart,'sec')
cv2.imwrite('k'+str(noOfClusters)+'_classical_kMeans_clusteredImage.png',clustered_image)

111 clusterHeads =  [39306, 25856, 25355, 38535]
iteration: 1 cluster building time: 26.031999349594116 sec
222 prevHeads =  [39306, 25856, 25355, 38535]
222 clusterHeads =  [47850, 46900, 67469, 67499]
iteration: 1 determining next centers time: 5.314569473266602 sec
iter_no 1 done
iteration: 2 cluster building time: 24.780426502227783 sec
222 prevHeads =  [47850, 46900, 67469, 67499]
222 clusterHeads =  [46945, 67195, 67468, 67498]
iteration: 2 determining next centers time: 5.255215406417847 sec
iter_no 2 done
iteration: 3 cluster building time: 28.67676615715027 sec
222 prevHeads =  [46945, 67195, 67468, 67498]
222 clusterHeads =  [67163, 67490, 67467, 67499]
iteration: 3 determining next centers time: 5.7821879386901855 sec
iter_no 3 done
iteration: 4 cluster building time: 28.81030297279358 sec
222 prevHeads =  [67163, 67490, 67467, 67499]
222 clusterHeads =  [67448, 67489, 67468, 67498]
iteration: 4 determining next centers time: 5.547331094741821 sec
iter_no 4 done
iteration: 5

True